<a href="https://colab.research.google.com/github/ParthG60/HousingMarket_St_Andrews/blob/main/VIP_calibration_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# testing minimization calibration


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math 
from scipy.optimize import minimize   #find a optimization routine: nlp,maix
from scipy.optimize import OptimizeResult
import pandas as pd
import sympy as sp
# ^^^ these are the relevant libraries and packages

Disutility_notSTA = 0 #disutility of not being in St Andrews
Num_UG= 8260 # Number of undegrards
Num_PG = 2164 # Number of postgrads
total_students = Num_UG+Num_PG
Prop_UG = Num_UG/total_students
Prop_PG = Num_PG/total_students
#NOTE: UGL - undergrad low income, PGL - postgrad low income and so on
# this naming convention is followed throughout the code

#Prop_UGL = 0.2 # Proportion of UGLs
#Prop_UGH = 1-Prop_UGL # Proportion of UGHs
#Prop_PGL = 0.3  # Proportion of PGLs
#Prop_PGH = 1-Prop_PGL #Proportion of PGHs
Quantity = 7000  # number of available rooms
e = math.e  # initialising eulers number
#R_guess = 500  # my guess for what equilibirum rent is

def PSA(utility,sig,R): # return probability of being in STA based on the sigmoid function
  return  (1-1 / (1 + e**((utility - Disutility_notSTA -R)/sig)))
  #implicit assumption made here that the error is logistically distributed

def demand_UGL(Prop_UGL,WTP_UGL,sig,R):   # calculates demand for Undergrad Low income
    UGL = Prop_UGL*Num_UG*(PSA(WTP_UGL,sig,R))
    return UGL
def demand_UGH(Prop_UGL,WTP_UGH,sig,R):    # calculates demand for Undergrad High income
    UGH = (1-Prop_UGL)*Num_UG*(PSA(WTP_UGH,sig,R))
    return UGH
def demand_PGL(Prop_PGL,WTP_PGL,sig,R):  # calculates demand for Postgrad Low income
    PGL = Prop_PGL*Num_PG*(PSA(WTP_PGL,sig,R))
    return PGL
def demand_PGH(Prop_PGL,WTP_PGH,sig,R):  # calculates demand for Postgrad high income
    PGH = (1-Prop_PGL)*Num_PG*(PSA(WTP_PGH,sig,R))
    return PGH



In [ ]:
def constraint_UG(parameters):    #inequality
    (Prop_UGL,Prop_PGL, WTP_UGL, WTP_UGH, WTP_PGL, WTP_PGH, R,sig) = parameters
    return WTP_UGH-WTP_UGL-100

def constraint_PG(parameters):   #inequality
    (Prop_UGL,Prop_PGL, WTP_UGL, WTP_UGH, WTP_PGL, WTP_PGH, R,sig) = parameters
    return WTP_PGH-WTP_PGL-100

def constraint_demand(parameters): 
    (Prop_UGL,Prop_PGL, WTP_UGL, WTP_UGH, WTP_PGL, WTP_PGH, R,sig) = parameters
    total_demand = demand_UGL(Prop_UGL,WTP_UGL,sig,R)+demand_UGH(Prop_UGL,WTP_UGH,sig,R)+demand_PGL(Prop_PGL,WTP_PGL,sig,R)
    +demand_PGH(Prop_UGL,WTP_PGH,sig,R)
    return total_demand-Quantity

def constraint_elasticity(parameters): 
    (Prop_UGL,Prop_PGL, WTP_UGL, WTP_UGH, WTP_PGL, WTP_PGH, R,sig) = parameters

    total_demand = demand_UGL(Prop_UGL,WTP_UGL,sig,R)+demand_UGH(Prop_UGL,WTP_UGH,sig,R)+demand_PGL(Prop_PGL,WTP_PGL,sig,R)
    +demand_PGH(Prop_UGL,WTP_PGH,sig,R)

    total_demand_2 = demand_UGL(Prop_UGL,WTP_UGL,sig,R+1)+demand_UGH(Prop_UGL,WTP_UGH,sig,R+1)+demand_PGL(Prop_PGL,WTP_PGL,sig,R+1)
    +demand_PGH(Prop_UGL,WTP_PGH,sig,R+1)

    return (total_demand_2-total_demand)*R/(total_demand) + 0.76

def objective(parameters): # returns all the relevant theoretical moments and parameters to fsolve
    (Prop_UGL,Prop_PGL, WTP_UGL, WTP_UGH, WTP_PGL, WTP_PGH, R,sig) = parameters
    
    total_demand = demand_UGL(Prop_UGL,WTP_UGL,sig,R)+demand_UGH(Prop_UGL,WTP_UGH,sig,R)+demand_PGL(Prop_PGL,WTP_PGL,sig,R) 
    +demand_PGH(Prop_UGL,WTP_PGH,sig,R)
    total_demand_2 = demand_UGL(Prop_UGL,WTP_UGL,sig,R+1)+demand_UGH(Prop_UGL,WTP_UGH,sig,R+1)+demand_PGL(Prop_PGL,WTP_PGL,sig,R+1)
    +demand_PGH(Prop_UGL,WTP_PGH,sig,R+1)

    Proportion_UGL = demand_UGL(Prop_UGL,WTP_UGL,sig,R)/(Prop_UGL*Num_UG) - 0.6
    Proportion_UGH = demand_UGH(Prop_UGL,WTP_UGH,sig,R)/((1-Prop_UGL)*Num_UG) - 0.8
    Proportion_PGL = demand_PGL(Prop_PGL,WTP_UGH,sig,R)/(Prop_PGL*Num_PG) - 0.5
    Proportion_PGH = demand_PGH(Prop_PGL,WTP_PGH,sig,R)/((1-Prop_PGL)*Num_PG) - 0.7
    WTP_difference = WTP_UGL - WTP_UGH - WTP_PGL + WTP_PGH  # should ditch this 
    # need to add equilibrium 
    # 584.32 - In StA
    # 480  - Out StA
    Rent = R - 580
    Market_clearing = total_demand-Quantity   # remove quantity as it's not independant
    elasticity_D = (total_demand_2-total_demand)*R/(total_demand) + 0.76
    #Rent = (Prop_UGL*WTP_UGL +(1-Prop_UGL*WTP_UGH))*Prop_UG + (Prop_PGL*WTP_PGL +(1-Prop_PGL*WTP_PGH))*Prop_PG - 600  #DOUBT ABOUT THIS
    error_sum = (Prop_UGL-0.5)**2 + (Prop_PGL-0.5)**2
    list_of_equations = [Proportion_UGL,Proportion_UGH,Proportion_PGL,Proportion_PGH,WTP_difference,Market_clearing/7000**0.5,elasticity_D, Rent/600**0.5]
    for i in list_of_equations:
      error_sum = error_sum + i**2
    return error_sum# returning the list of equations

    

In [ ]:
prop_limit = (0.02,0.95)
WTP_limit = (350,1200)
rent_limit = (450,750)
sig_limit = (2,1000)

limits = (prop_limit,prop_limit,WTP_limit,WTP_limit,WTP_limit,WTP_limit,rent_limit,sig_limit)

con1 = {'type':'ineq','fun':constraint_UG}
con2 = {'type':'ineq','fun':constraint_PG}
#con4 = {'type':'eq','fun':constraint_WTP2}
con5 = {'type':'eq','fun':constraint_demand}
con6 = {'type':'eq','fun':constraint_elasticity}

cons = [con1,con2,con5,con6]
#print it out and debug the code
#some places might not have the correct output 
# very interestig result through this combination [0.1,0.2,500,550,525,575,20,500]
Guess = [0.1,0.2,500,550,525,575,20,500] #Prob_UGL,Prob_PGL, WTP_UGL, WTP_UGH,WTP_PGL,WTP_PGH, R, sig

Results = minimize(objective,Guess,method='SLSQP', bounds=limits,constraints=cons)  # fsolve gives us the actual root regardless of the guess
Results

#(Prop_UGL,Prop_PGL, WTP_UGL, WTP_UGH, WTP_PGL, WTP_PGH, R,sig)


     fun: 0.7158289138728645
     jac: array([-6.52876563e-01, -1.84217393e-02, -5.57412207e-03, -6.46978617e-04,
       -6.46978617e-04,  6.47448003e-04,  5.78711927e-03, -1.23312697e-02])
 message: 'Optimization terminated successfully.'
    nfev: 214
     nit: 21
    njev: 21
  status: 0
 success: True
       x: array([1.73722818e-01, 4.91432248e-01, 5.47418816e+02, 8.45956162e+02,
       4.67125043e+02, 7.65662642e+02, 5.79893696e+02, 1.63891662e+01])

In [ ]:
def total_demand(Prop_UGL,Prop_PGL, WTP_UGL, WTP_UGH, WTP_PGL, WTP_PGH, R,sig):
    total = (demand_UGL(Prop_UGL,WTP_UGL,sig,R)+demand_UGH(Prop_UGL,WTP_UGH,sig,R)+demand_PGL(Prop_PGL,WTP_PGL,sig,R)+demand_PGH(Prop_UGL,WTP_PGH,sig,R))
    return total
total_demand(1.73711735e-01, 4.93145402e-01, 5.47448811e+02, 8.45951944e+02,
       4.67236793e+02, 7.65740121e+02, 5.79919399e+02, 1.63822990e+01)


In [ ]:
##demand_bin(Prop_binL,WTP_bin,sig,R)
## Results in order of  Prop_UGL,Prop_PGL, WTP_UGL, WTP_UGH, WTP_PGL, WTP_PGH, R,sig
end_UGL = demand_UGL(Results.x[0], Results.x[2], Results.x[7], Results.x[6])
end_UGH = demand_UGH(Results.x[0], Results.x[3], Results.x[7], Results.x[6])
end_PGL = demand_PGL(Results.x[1], Results.x[4], Results.x[7], Results.x[6])
end_PGH = demand_PGH(Results.x[1], Results.x[5], Results.x[7], Results.x[6])
print(end_UGL+end_UGH+end_PGL+end_PGH)

a = total_demand(Results.x[0], Results.x[1], Results.x[2], Results.x[3], Results.x[4], Results.x[5], Results.x[6], Results.x[7])
print(a)

6929.365498749628
7000.000000023251


In [ ]:
print(end_PGL/total_students)
print(end_PGH/total_students)

0.05830654123988201
0.06359213107248532


In [ ]:
##outputs

##proportions, WTP, number in (proportion*total # students)
x = {'UGL': [np.round(end_UGL/total_students, decimals=4), np.round(end_UGL/a, decimals=4), np.round(Results.x[2], decimals=2), np.round(end_UGL, decimals=0)], 
     'UGH': [(np.round(end_UGH/total_students, decimals=4)), np.round(end_UGH/a, decimals=4), np.round(Results.x[3], decimals=2), np.round(end_UGH, decimals=0)], 
     'PGL': [np.round(end_PGL/total_students, decimals=4), np.round(end_PGL/a, decimals=4), np.round(Results.x[4], decimals=2), np.round(end_PGL, decimals=0)], 
     'PGH': [(np.round(end_PGH/total_students, decimals=4)), np.round(end_PGH/a, decimals=4), np.round(Results.x[5], decimals=2), np.round(end_PGH, decimals=0)], 
     'Total':['N/A', 'N/A', np.round(Results.x[6], decimals=2), np.round(a)]}
index1 = ['Proportion of Total Students Living in St. Andrews for Each Bin', 'Proportion of St Andrews Rooms Let to Each Bin', 'Willingness to Pay/Equilibrium Rent', 'Number of Students Living in St Andrews']

df = pd.DataFrame(data=x, index=index1)
df

,UGL,UGH,PGL,PGH,Total
Proportion of Total Students Living in St. Andrews for Each Bin,0.237,0.3058,0.0583,0.0636,N/A
Proportion of St Andrews Rooms Let to Each Bin,0.353,0.4554,0.0868,0.0947,N/A
Willingness to Pay/Equilibrium Rent,724.600,824.6000,621.6300,721.6300,579.98
Number of Students Living in St Andrews,2471.000,3188.0000,608.0000,663.0000,7000.0


# testing draft 5


Changes made: 


*   switch R and sig guesses
*   calculate proportion_bin with PSA



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math
from scipy.optimize import fsolve
import pandas as pd
import sympy as sp
# ^^^ these are the relevant libraries and packages


In [ ]:

Disutility_notSTA = -400 #disutility of not being in St Andrews
Num_UG= 8260 # Number of undegrards
Num_PG = 2164 # Number of postgrads

Quantity = 7000  # number of available rooms
e = math.e  # initialising eulers number


In [ ]:
total_students = Num_UG+Num_PG
Prop_UG = Num_UG/total_students
Prop_PG = Num_PG/total_students


In [ ]:
def PSA(utility,sig,R): # return probability of being in STA based on the sigmoid function
  return  (1-(1 / (1 + e**((utility - Disutility_notSTA -R)/sig))))
  #implicit assumption made here that the error is logistically distributed


In [ ]:
PSA(5.00e+02,2.00e+01, 5.00e+02 ) ##equlibrium probability of being in STA for UGL???

0.9999999979388464

In [ ]:
def demand_UGL(Prop_UGL,WTP_UGL,sig,R):   # calculates demand for Undergrad Low income
    UGL = Prop_UGL*Num_UG*(PSA(WTP_UGL,sig,R))
    return UGL


In [ ]:
def demand_UGH(Prop_UGL,WTP_UGH,sig,R):    # calculates demand for Undergrad High income
    UGH = (1-Prop_UGL)*Num_UG*(PSA(WTP_UGH,sig,R))
    return UGH


In [ ]:
def demand_PGL(Prop_PGL,WTP_PGL,sig,R):  # calculates demand for Postgrad Low income
    PGL = Prop_PGL*Num_PG*(PSA(WTP_PGL,sig,R))
    return PGL


In [ ]:
def demand_PGH(Prop_PGL,WTP_PGH,sig,R):  # calculates demand for Postgrad high income
    PGH = (1-Prop_PGL)*Num_PG*(PSA(WTP_PGH,sig,R))
    return PGH


In [ ]:


def equations(parameter): # returns all the relevant theoretical moments and parameters to fsolve
    (Prop_UGL,Prop_PGL, WTP_UGL, WTP_UGH, WTP_PGL, WTP_PGH, R,sig) = parameter
    
    total_demand = demand_UGL(Prop_UGL,WTP_UGL,sig,R)+demand_UGH(Prop_UGL,WTP_UGH,sig,R)+demand_PGL(Prop_PGL,WTP_PGL,sig,R)
    +demand_PGH(Prop_UGL,WTP_PGH,sig,R)
    total_demand_2 = demand_UGL(Prop_UGL,WTP_UGL,sig,R+1)+demand_UGH(Prop_UGL,WTP_UGH,sig,R+1)+demand_PGL(Prop_PGL,WTP_PGL,sig,R+1)
    +demand_PGH(Prop_UGL,WTP_PGH,sig,R+1)

    ##Proportion_UGL = demand_UGL(Prop_UGL,WTP_UGL,sig,R)/(Prop_UGL*Num_UG) - 0.7
    ##Proportion_UGH = demand_UGH(Prop_UGL,WTP_UGH,sig,R)/((1-Prop_UGL)*Num_UG) - 0.9
    ##Proportion_PGL = demand_PGL(Prop_PGL,WTP_UGH,sig,R)/(Prop_PGL*Num_PG) - 0.6
    ##Proportion_PGH = demand_PGH(Prop_PGL,WTP_PGH,sig,R)/((1-Prop_PGL)*Num_PG) - 0.8
    Proportion_UGL = PSA(WTP_UGL, sig, R) - 0.7
    Proportion_UGH = PSA(WTP_UGH, sig, R) -0.9
    Proportion_PGL = PSA(WTP_PGL, sig, R) - 0.6
    Proportion_PGH = PSA(WTP_PGL, sig, R) - 0.8
    WTP_difference = WTP_UGL - WTP_UGH - WTP_PGL + WTP_PGH  # should ditch this 
    # need to add equilibrium 
    # 584.32 - In StA
    # 480  - Out StA
    Rent = R - 580
    Market_clearing = total_demand-Quantity   # remove quantity as it's not independant
    elasticity_D = (total_demand_2-total_demand)*R/(total_demand) - 0.76
    ##Rent = (Prop_UGL*WTP_UGL +(1-Prop_UGL*WTP_UGH))*Prop_UG + (Prop_PGL*WTP_PGL +(1-Prop_PGL*WTP_PGH))*Prop_PG - 600  #DOUBT ABOUT THIS

    list_of_equations = [Proportion_UGL,Proportion_UGH,Proportion_PGL,Proportion_PGH,WTP_difference,Market_clearing,elasticity_D, Rent]
    return list_of_equations# returning the list of equations

#print it out and debug the code
#some places might not have the correct output
 



In [ ]:
# very interestig result through this combination [0.1,0.2,500,550,525,575,20,500]
Guess = [0.1,0.2,500,550,525,575,500,20] #Prob_UGL,Prob_PGL, WTP_UGL, WTP_UGH,WTP_PGL,WTP_PGH, R, sig
Results = fsolve(equations ,Guess)  # fsolve gives us the actual root regardless of the guess.
print(Results)


[1.00e-01 2.00e-01 5.00e+02 5.50e+02 5.25e+02 5.75e+02 5.00e+02 2.00e+01]


/usr/local/lib/python3.7/dist-packages/scipy/optimize/minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


In [ ]:
Results[6]

500.0

# #2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math
from scipy.optimize import fsolve
import pandas as pd
# ^^^ these are the relevant libraries and packages


##trying to calibrate these, so don't run them
##Prop_UGL = 0.2 # Proportion of UGLs
##Prop_UGH = 1-Prop_UGL # Proportion of UGHs
##Prop_PGL = 0.3  # Proportion of PGLs
##Prop_PGH = 1-Prop_PGL #Proportion of PGHs

##^^these seem a little weird, are we sure those are even close 
##if changed, remember to change in the guess array as well





Disutility_notSTA = -400 #disutility of not being in St Andrews
I_low = 20000  #low income
I_high = 40000+10000 #high income
D_UG = 1  # degree UG = 1
D_PG = 0 #degree PG = 0
Num_UG= 8260 # Number of undegrards
Num_PG = 2164 # Number of postgrads

##add these back in later
##Prop_UG_L = Num_UG      ##proportion of total undergrads that are low income
##Prop_PG_L

#NOTE: UGL - undergrad low income, PGL - postgrad low income and so on
# this naming convention is followed throughout the code

Quantity = 7000  # number of available rooms
e = math.e  # initialising eulers number
##R_guess = 666  # my guess for what equilibirum rent is
##^^I took out R_guess because we're not calibrating R right now, correct?



def PSA(utility, sig): # return probability of being in STA based on the sigmoid function
  return  (1-1 / (1 + e**((utility - Disutility_notSTA)/sig)))
  #implicit assumption made here that the error is logistically distributed

#NOTE: from here on out "R" is used in several functions
# R is Equilibrium Rent we are trying to solve for
def demand_UGL(R, WTP_UGL, Prop_UGL, sig):   # calculates demand for Undergrad Low income
    UGL = Prop_UGL*Num_UG*(PSA(WTP_UGL, sig))
    return UGL
def demand_UGH(R, WTP_UGH, Prop_UGL, sig):    # calculates demand for Undergrad High income
    UGH = (1-Prop_UGL)*Num_UG*(PSA(WTP_UGH, sig))
    return UGH
def demand_PGL(R, WTP_PGL, Prop_PGL, sig):  # calculates demand for Postgrad Low income
    PGL = Prop_PGL*Num_PG*(PSA(WTP_PGL, sig))
    return PGL
def demand_PGH(R, WTP_PGH, Prop_PGL, sig):  # calculates demand for Postgrad high income
    PGH = (1-Prop_PGL)*Num_PG*(PSA(WTP_PGH, sig))
    return PGH

def total_demand(R, WTP_UGL, WTP_UGH, WTP_PGL, WTP_PGH, Prop_UGL, Prop_PGL, sig): # sums all the other demand to give total demand
    total_demand = demand_UGL(R, WTP_UGL, Prop_UGL, sig)+demand_UGH(R, WTP_UGH, Prop_UGL, sig)+demand_PGL(R, WTP_PGL, Prop_PGL, sig)+demand_PGH(R, WTP_PGH, Prop_PGL, sig)
    return total_demand-Quantity # returning the total demand - Quantity
    # We subtract Quantity here because fsolve solves for the roots



##this is where I put the seven independent theoretical moments 
R = 658 ##setting equilibrium average rent constant
Prop_in_UGL = np.round(demand_UGL(R, WTP_UGL)/(Prop_UGL*Num_UG))
Prop_in_UGL = 0.8    ##need the constant for all from data team Prop_in_UGL 
Prop_in_UGH = np.round(demand_UGH(R, WTP_UGH)/(Prop_UGH*Num_UG) )
Prop_in_UGH = 1   ##Prop_in_UGH
Prop_in_PGL =np.round(demand_PGL(R, WTP_PGL)/(Prop_PGL*Num_PG)) 
Prop_in_PGL = 0.4   ##Prop_in_PGL 
Prop_in_PGH = np.round(demand_PGH(R, WTP_PGH)/(Prop_PGH*Num_PG)) 
Prop_in_PGH = 0.7   ##Prop_in_PGH 

premium = abs(WTP_UGH-WTP_UGL) - abs(WTP_PGH-WTP_PGL)
premium = 0
elasticity_D = (R/total_demand(R))*(sp.diff(total_demand(R))) 
elasticity_D = 0.76   ##elasticity constant from last year's report


##calibration, starting just w/ willingness to pay

##after we figure out WTP, we can add in var_error and proportions
Var_error = variance(sig)
##I have no idea how to add in the proportion of undergrads 
##that are low income and proportion of postgrads that are low income


array1 = np.array([WTP_UGL, WTP_UGH, WTP_PGL, WTP_PGH, Var_error, Prop_UGL, Prop_PGL])

array2 = np.array([400, 700, 300, 600, 20, 0.2, 0.3]) ##guesses
R = fsolve(array1,array2)  # fsolve gives us the actual root regardless of the guess.



NameError: ignored

# #1

In [ ]:
##utilities from the prevous model
coef_I = 7/5000   # Income coefficient
coef_R = 1 # Rent coefficient
coef_D = 40  # degree coefficient
Disutility_notSTA = -400 #disutility of not being in St Andrews
I_low = 20000  #low income
I_high = 40000+10000 #high income
D_UG = 1  # degree UG = 1
D_PG = 0 #degree PG = 0
Num_UG= 8260 # Number of undegrards
Num_PG = 2164 # Number of postgrads


u_UGL = coef_I*I_low-coef_R*R+coef_D*D_UG
u_UGH = coef_I*I_high-coef_R*R+coef_D*D_UG
u_PGL = coef_I*I_low-coef_R*R+coef_D*D_PG
u_PGH = coef_I*I_high-coef_R*R+coef_D*D_PG


NameError: ignored

In [ ]:
##we still don't have the data to actually complete it, but there's still a lot to do here
##mostly because I'm not 100% sure what I'm doing

import matplotlib.pyplot as plt
import numpy as np
import math
from scipy.optimize import fsolve
import pandas as pd
# ^^^ these are the relevant libraries and packages



## can't assign sig this value now, sig = 20 #variance/SD  for error term   .
##check my math
sig = (utility-Disutility_notSTA)/np.log(np.exp(Prob_in))
##Prob_in is the probability of being in town as calculated by the PSA, I'm not sure how this is going to work, I definetly messed up

Disutility_notSTA = -400 #disutility of not being in St Andrews
I_low = 20000  #low income
I_high = 40000+10000 #high income
D_UG = 1  # degree UG = 1
D_PG = 0 #degree PG = 0
Num_UG= 8260 # Number of undegrards
Num_PG = 2164 # Number of postgrads

#NOTE: UGL - undergrad low income, PGL - postgrad low income and so on
# this naming convention is followed throughout the code


##remove the props so we can solve for that later. We still need to define them but I'm not sure how to go about doing that
        ##Prop_UGL = 0.2 # Proportion of UGLs
        ##Prop_UGH = 1-Prop_UGL # Proportion of UGHs
        ##Prop_PGL = 0.3  # Proportion of PGLs
        ##Prop_PGH = 1-Prop_PGL #Proportion of PGHs
Quantity = 7000  # number of available rooms
e = math.e  # initialising eulers number
R_guess = 500   # my guess for what equilibirum rent is

def PSA(utility): # return probability of being in STA based on the sigmoid function
  return  (1-1 / (1 + e**((utility - Disutility_notSTA)/sig)))
  #implicit assumption made here that the error is logistically distributed

#NOTE: from here on out "R" is used in several functions
# R is Equilibrium Rent we are trying to solve for
def demand_UGL(R, Prop_UGL):   # calculates demand for Undergrad Low income
    UGL = Prop_UGL*Num_UG*(PSA(coef_I*I_low-coef_R*R+coef_D*D_UG))
    return UGL
def demand_UGH(R, Prop_UGL):    # calculates demand for Undergrad High income
    Prop_UGH = 1-Prop_UGL
    UGH = Prop_UGH*Num_UG*(PSA(coef_I*I_high-coef_R*R+coef_D*D_UG))
    return UGH
def demand_PGL(R, Prop_PGL):  # calculates demand for Postgrad Low income
    PGL = Prop_PGL*Num_PG*(PSA(coef_I*I_low-coef_R*R+coef_D*D_PG))
    return PGL
def demand_PGH(R, Prop_PGL):  # calculates demand for Postgrad high income
    Prop_PGH = 1-Prop_PGL
    PGH = Prop_PGH*Num_PG*(PSA(coef_I*I_high-coef_R*R+coef_D*D_PG))
    return PGH

def total_demand(R): # sums all the other demand to give total demand
    total_demand = demand_UGL(R)+demand_UGH(R)+demand_PGL(R)+demand_PGH(R)
    return total_demand-Quantity # returning the total demand - Quantity
    # We subtract Quantity here because fsolve solves for the roots


##guesses
guess_WTP_UGL = 500  ##I totally made this up
guess_WTP_UGH = 700
guess_WTP_PGL = 500
guess_WTP_PGH = 700
guess_Prop_UG_L = .2
guess_Prop_PG_L = .3
guess_Var_error = 20


array1 = np.array([total_demand, Prop_UGL, Prop_PGL])
array2 = np.array([500, .2, .3])
R = fsolve(array1,array2)  # fsolve gives us the actual root regardless of the guess.
# the guess is just used to hint at which side of the root we may want -ve or +ve
print("Equilibrium Rent :" +str(R))


NameError: ignored

In [ ]:
##this is where I put the seven independent theoretical moments 
R = 658 ##setting equilibrium average rent constant
np.round(demand_UGL(R)/(Prop_UGL*Num_UG) = 0.8 ##need the constant for all from data team Prop_in_UGL 
np.round(demand_UGH(R)/(Prop_UGH*Num_UG) = 1   ##Prop_in_UGH
np.round(demand_PGL(R)/(Prop_PGL*Num_PG) = 0.4   ##Prop_in_PGL 
np.round(demand_PGH(R)/(Prop_PGH*Num_PG) = 0.7   ##Prop_in_PGH 
abs(WTP_UGH-WTP_UGL) = abs(WTP_PGH-WTP_PGL)
(R/total_demand(R))*(sp.diff(total_demand(R))) = 0.76   ##elasticity constant from last year's report

SyntaxError: ignored

In [ ]:
##guesses
guess_WTP_UGL = 500  ##I totally made this up
guess_WTP_UGH = 700
guess_WTP_PGL = 500
guess_WTP_PGH = 700
guess_Prop_UG_L = .2
guess_Prop_PG_L = .3
guess_Var_error = 20

In [ ]:
##x0 gives the guess to the fsolve
x0 = ([guess_WTP_UGL, guess_WTP_UGH, guess_WTP_PGL, guess_WTP_PGH, guess_Prop_UG_L, guess_Prop_PG_L, guess_Var_error])
def calibrate(x): ## this is where I put the seven parameters and return as an array
  WTP_UGL =  ##equation =0 because fsolve finds roots, but I'm not sure what that equation should be
  WTP_UGH
  WTP_PGL
  WTP_PGH
  Prop_UG_L = Num_UG      ##proportion of total undergrads that are low income
  Prop_PG_L
  Var_error = variance(sig)
  return np.array([WTP_UGL, WTP_UGH, WTP_PGL, WTP_PGH, Prop_UG_L, Prop_PG_L, Var_error])


x = fsolve(calibrate, x0) ##should = 0


In [ ]:
##results
WTP_UGL = x[0]
WTP_UGH = x[1]
WTP_PGL = x[2]
WTP_PGH = x[3]
Prop_UGL_in = x[4]
Prop_PGL_in = x[5]
Var_error = x[6]
print(WTP_UGL, WTP_UGH, WTP_PGL, WTP_PGH, Prop_UGL_in, Prop_PGL_in, Var_error, calibrate(x))

